# config file into solara web app

In [1]:
import solara
from ipyleaflet import Map, DrawControl, basemaps, Marker, Icon
from ipywidgets import Output, Dropdown, FloatSlider, Text
from IPython.display import display
import json
from pathlib import Path
import datetime as dt
import os

In [2]:
# HERE: DROP-DOWN MENU; people won't have datasets on their local drive

FA_databases = ["Scheveningen", "Option B", "Option C"] #################################
FA_database = solara.reactive("Scheveningen")


@solara.component
def Page():
    solara.Select(label="FloodAdapt data base", value=FA_database, values=FA_databases)
    
    solara.Markdown(f"**Selected**: {FA_database.value}")


########################################################################################################################################################################################
# The following line is required only when running the code in a Jupyter notebook:
#Page()

In [3]:
if FA_database.value == "Scheveningen":
    fa_database = Path(r"/mnt/c/Users/santjer/OneDrive - Stichting Deltares/Documents/DestinE/Technical/01_FloodAdapt/scheveningen")
elif FA_database.value == "Option B":
    fa_database = Path(r"/mnt/c/Users/santjer/OneDrive - Stichting Deltares/Documents/DestinE/Technical/01_FloodAdapt/OPTIONB/") #########################
else:
    fa_database = Path(r"/mnt/c/Users/santjer/OneDrive - Stichting Deltares/Documents/DestinE/Technical/01_FloodAdapt/OPTIONC/") #########################


# HERE: whatever the solution is; people won't have datasets on their local drive

# Path to data folder. If (relative) file paths are used when specifying data sets, it will be looked for in this folder.
# data_folder = Path("/home/wotromp/InterTwin/Data")  


# OPTIONAL: HydroMT data catalogue from which to fetch data. The order matters here, as datasets are looked for in a catalogue in order from first catalogue name in list to last.
data_catalogues = [Path(r"/mnt/c/Users/santjer/OneDrive - Stichting Deltares/Documents/DestinE/Technical/01_FloodAdapt/deltares_data_wsl.yml")]

# Page map and tabs

In [ ]:
# >>>>>>>>>>>>>>>> CORRECT BEST VERSION OF CODE BELOW \_v_/ <<<<<<<<<<<<<<<<<<<<<
polyline_coordinates = solara.reactive([])
polygon_coordinates = solara.reactive([])
marker_coordinates = solara.reactive([])
drawn_polylines = solara.reactive([])
drawn_polygons = solara.reactive([])
drawn_markers = solara.reactive([])
selected_geometry = solara.reactive(None) 
selected_tab = solara.reactive('Event') 
measures_list = solara.reactive([])
error_message = solara.reactive("")
current_geometry_type = solara.reactive(None)  # Track current geometry type
point_placed = solara.reactive(False)
open_delete_confirmation = solara.reactive(False)

m = Map(center=(52.08654741528378, 4.295223531699989), zoom=10, scroll_wheel_zoom=True, basemap=basemaps.OpenStreetMap.Mapnik)

draw_control = DrawControl(
    polyline={'shapeOptions': {'color': 'blue', 'weight': 4}},
    polygon={'shapeOptions': {'color': 'red', 'weight': 4}},
    marker={},  # Allow marker drawing
    rectangle={},  # Disables rectangle drawing
    circle={},  # Disables circle drawing
)

def handle_draw(target, action, geo_json):
    print(f"Action: {action}, GeoJSON: {geo_json}") 
    if action == 'created' and 'geometry' in geo_json:
        geo_type = geo_json['geometry']['type']
        coords = geo_json['geometry']['coordinates']
        
        if current_geometry_type.value and current_geometry_type.value != geo_type:
            error_message.value = f"**Error**: You cannot draw a {geo_type} after a {current_geometry_type.value}. Please clear the previous geometry before adding a new type."
            return
        
        if geo_type == 'Polygon':
            polygon_coords = coords[0]
            selected_geometry.value = {'type': 'Polygon', 'coordinates': polygon_coords}
            drawn_polygons.value.append({'type': 'Polygon', 'coordinates': polygon_coords})
            print("Polygon coordinates updated:", selected_geometry.value)
        
        elif geo_type == 'LineString':  # Handling polylines
            selected_geometry.value = {'type': 'LineString', 'coordinates': coords}
            drawn_polylines.value.append({'type': 'LineString', 'coordinates': coords})
            print("Polyline coordinates updated:", selected_geometry.value)

        elif geo_type == 'Point':  # Handling markers
            if point_placed.value:
                error_message.value = "**Error**: Only one point can be placed. Please first save the measure before adding another one."
                return
            selected_geometry.value = {'type': 'Point', 'coordinates': coords}
            drawn_markers.value.append({'type': 'Point', 'coordinates': coords})
            point_placed.set(True)  # Set point placed flag to True
            print("Marker coordinates updated:", selected_geometry.value)


        current_geometry_type.value = geo_type  # Update the current geometry type
        error_message.value = ""


draw_control.on_draw(handle_draw)
m.add_control(draw_control)


map_output = Output()
with map_output:
    display(m)


####################################### TAB 1 WEATHER EVENT #######################################
selected_start_date = dt.date.today()
selected_end_date = dt.date.today()
eventName = solara.reactive("Event Name")
event_dict = {}

@solara.component
def TabEvent():
    global selected_start_date, selected_end_date
    start_date = solara.use_reactive(dt.date.today())
    end_date = solara.use_reactive(dt.date.today())
    selected_start_date = start_date.value
    selected_end_date = end_date.value
    
    def save_inputs():
        event_dict["name"] = eventName.value
        event_dict["start_time"] = start_date.value.strftime('%Y-%m-%d %H:%M:%S')
        event_dict["end_time"] = end_date.value.strftime('%Y-%m-%d %H:%M:%S')
        event_dict["data_catalogues"] = 'MISSING'
        event_dict["sfincs_forcing"] = {'meteo': 'MISSING', 'waterlevel': 'MISSING'}

    with solara.Card("Configure the Weather Event", style={"width": "100%", "padding": "10px"}):
        solara.InputText("Event Name", value=eventName, continuous_update=True)
        solara.Markdown(f"**Your Event Name**: {eventName.value}")
        
        solara.Text("Select the Start Date:")
        solara.lab.InputDate(start_date)
        
        solara.Text("Select the End Date:")
        solara.lab.InputDate(end_date)
        
        if end_date.value < start_date.value:
            solara.Markdown("**Warning**: The end date cannot be earlier than the start date.", style={"color": "red"})
        
        solara.Button("Save Inputs", on_click=save_inputs)


####################################### TAB 2 PROJECTIONS #######################################
SLR_input = solara.reactive(0.0)
RAIN_input = solara.reactive(0.0)
POP_input = solara.reactive(0.0)
ECON_input = solara.reactive(0.0)
projName = solara.reactive("Projection Name")
proj_dict = {}

@solara.component
def TabProjections():
    def save_inputs_projections():
        proj_dict["name"] = projName.value
        proj_dict["physical_projection"] = {'sea_level_rise': SLR_input.value,
                                            'rainfall_increase': RAIN_input.value}
        proj_dict["socio_economic_change"] = {'population_growth_existing': POP_input.value,
                                            'economic_growth': ECON_input.value}


    with solara.Card("Projections", style={"width": "100%", "padding": "10px"}):
        solara.InputText("Projection Name", value=projName, continuous_update=True)
        solara.Markdown(f"**Your Projection Name**: {projName.value}")
        solara.InputFloat("Sea Level Rise (metres)", value=SLR_input, continuous_update=True)
        solara.InputFloat("Precipitation Increase (%)", value=RAIN_input, continuous_update=True)
        solara.InputFloat("Population Growth (%)", value=POP_input, continuous_update=True)
        solara.InputFloat("Economic Growth (%)", value=ECON_input, continuous_update=True)
        
        solara.Button("Save Inputs", on_click=save_inputs_projections, style={"margin-top": "20px"})


####################################### TAB 3 MEASURES #######################################
strategyName = solara.reactive("Strategy Name")
measureName = solara.reactive("Measure Name")
measure_types = ["Floodwall","Pump","Water square","Green infrastructure","Water storage",
                 "Elevate properties","Floodproof properties","Buyout properties"]
measure_type = solara.reactive("Floodwall")
elevation_fw = solara.reactive(0)  # For floodwall 
discharge_fw = solara.reactive(0)  # For pump 
volume_ws = solara.reactive(0)  # For water square
height_ws = solara.reactive(0)  # For water square 
volume_gi = solara.reactive(0)  # For green infrastructure
height_gi = solara.reactive(0)  # For green infrastructure
percarea_gi = solara.reactive(0)  # For green infrastructure 
volume_wst = solara.reactive(0)  # For water storage 
elevation_prop = solara.reactive(0)  # For properties 
elevation_fpprop = solara.reactive(0)  # For floodproof properties 
strategy = {} 

def clear_all_drawn_geometries():
    global draw_control
    m.remove_control(draw_control)
    draw_control = DrawControl(
        polyline={'shapeOptions': {'color': 'blue', 'weight': 4}},
        polygon={'shapeOptions': {'color': 'red', 'weight': 4}},
        marker={},
        rectangle={},
        circle={},    )
    draw_control.on_draw(handle_draw)
    m.add_control(draw_control)
    drawn_polylines.set([])
    drawn_polygons.set([])
    drawn_markers.set([])
    point_placed.set(False) 

def update_draw_tools():
    print(f"Updating draw tools for measure_type: {measure_type.value}")
    global draw_control
    m.remove_control(draw_control)
    
    if measure_type.value == "Floodwall":
        new_draw_control = DrawControl(polyline={'shapeOptions': {'color': 'blue', 'weight': 4}},polygon={},circlemarker={})
    elif measure_type.value == "Pump":
        new_draw_control = DrawControl(polyline={},polygon={},marker={})  # Enable marker only
    elif measure_type.value in ["Water square", "Green infrastructure", "Water storage", "Elevate properties", "Floodproof properties", "Buyout properties"]:
        new_draw_control = DrawControl(polyline={},polygon={'shapeOptions': {'color': 'red', 'weight': 4}},circlemarker={})

    draw_control = new_draw_control
    draw_control.on_draw(handle_draw)
    m.add_control(draw_control)

    drawn_polylines.set([])
    drawn_polygons.set([])
    drawn_markers.set([])
    point_placed.set(False)

def save_current_add_new():
        if selected_geometry.value is None:
            error_message.value = "**Error**: Please place the measure on the map."
            return
        else:
            error_message.set("")
        
        measure_misc  = {}
        if measure_type.value == "Floodwall":
            mtype = "floodwall"
            measure_misc = {"elevation": {"value": elevation_fw.value, "units": 'meters'}}
        elif measure_type.value == "Pump":
            mtype = "pump"
            measure_misc = {"discharge": {"value": discharge_fw.value, "units": 'm3/s'}}
        elif measure_type.value == "Water square":
            mtype = "water_square"
            measure_misc = {"volume": {"value": volume_ws.value, "units": 'm3'},
                            "height": {"value": height_ws.value, "units": "meters"}}
        elif measure_type.value == "Green infrastructure":
            mtype = "greening"
            measure_misc = {"volume": {"value": volume_gi.value, "units": 'm3'},
                            "height": {"value": height_gi.value, "units": "meters"},
                            "percent_area": percarea_gi.value}
        elif measure_type.value == "Water storage":
            mtype = "total_storage"
            measure_misc = {"volume": {"value": volume_wst.value, "units": 'm3'}}
        elif measure_type.value == "Elevate properties":
            mtype = "elevate_properties"
            measure_misc = {"elevation": {"value": elevation_prop.value, "units": 'meters'}}
        elif measure_type.value == "Floodproof properties":
            mtype = "floodproof_properties"
            measure_misc = {"elevation": {"value": elevation_fpprop.value, "units": 'meters'}}
        elif measure_type.value == "Buyout properties":
            mtype = "buyout_properties"
            measure_misc = {} # no additional properties

        measure = {
            "name": measureName.value,
            "type": mtype, # measure_type.value,
            "selection": selected_geometry.value,
            "misc": measure_misc        }
        measures_list.value.append(measure)
        save_directory = Path.cwd() # Path(r"/mnt/c/Users/santjer/OneDrive - Stichting Deltares/Documents/DestinE/Technical/02_Solara")
        geojson_data = {"measure_area": selected_geometry.value}
        geojson_filename = save_directory / f"{measureName.value}_geometry.geojson"
        with open(geojson_filename, 'w') as geojson_file:
            json.dump(geojson_data, geojson_file)
        measure["selection"] = str(geojson_filename) # - - - - - - - - - - - - - - - - - - - - - - - -
        clear_all_drawn_geometries()
        selected_geometry.set(None)
        current_geometry_type.set(None)
        measureName.set("Measure Name")
        measure_type.set("Floodwall")
        elevation_fw.set(0)
        discharge_fw.set(0) 
        volume_ws.set(0) 
        height_ws.set(0) 
        volume_gi.set(0) 
        height_gi.set(0) 
        percarea_gi.set(0)
        volume_wst.set(0) 
        elevation_prop.set(0) 
        elevation_fpprop.set(0)

def save_all_measures():
    global strategy
    error_message.set("")
    clear_all_drawn_geometries()
    selected_geometry.set(None)
    current_geometry_type.set(None)
    
    strategy = {'name': strategyName.value}
    for idx, measure in enumerate(measures_list.value, start=1):
        if measure['selection'] is None:
            continue
        measure_name = f"measure{idx}"
        file_path = Path.cwd() / f"{measure['name']}_geometry.geojson"# f"{measureName.value}_geometry.geojson" #####################
        strategy[measure_name] = {
            'name': measure['name'],
            'type': measure['type'].lower(),  # Assuming the type should be lowercase
            'selection': str(file_path), # str(geojson_filename),
            'misc': measure['misc']}
    strategyName.set("Strategy Name")

def clear_measures():
    global strategy
    for measure in measures_list.value:
        measure_name = measure['name']
        file_path = Path.cwd() / f"{measure_name}_geometry.geojson"
        if file_path.exists():
            file_path.unlink()
    measures_list.set([]) 
    selected_geometry.set(None)
    current_geometry_type.set(None)
    error_message.set("")
    clear_all_drawn_geometries()
    measureName.set("Measure Name")
    measure_type.set("Floodwall")
    strategyName.set("Strategy Name")
    strategy = {} 

def save_strategy():
    save_current_add_new()
    save_all_measures()


@solara.component
def TabMeasures():  

    with solara.Card("Strategy Name", style={"width": "100%", "padding": "10px"}):
        solara.InputText("Strategy Name", value=strategyName, continuous_update=True)
        
    with solara.Card("Measures", style={"width": "100%", "padding": "10px"}):
        solara.InputText("Measure Name", value=measureName, continuous_update=True)
        solara.Markdown(f"**Your Measure Name**: {measureName.value}")

        solara.Select(label="Measure Type", value=measure_type, values=measure_types)

        if measure_type.value == "Floodwall":
            solara.InputFloat("Elevation [m]", value=elevation_fw, continuous_update=True)
            solara.use_effect(lambda: update_draw_tools(), [measure_type.value])
        elif measure_type.value == "Pump":
            solara.InputFloat("Discharge [m³/s]", value=discharge_fw, continuous_update=True)
            solara.use_effect(lambda: update_draw_tools(), [measure_type.value])
        elif measure_type.value == "Water square":
            solara.InputFloat("Volume [m³]", value=volume_ws, continuous_update=True)
            solara.InputFloat("Height [m]", value=height_ws, continuous_update=True)
            solara.use_effect(lambda: update_draw_tools(), [measure_type.value])
        elif measure_type.value == "Green infrastructure":
            solara.InputFloat("Volume [m³]", value=volume_gi, continuous_update=True)
            solara.InputFloat("Height [m]", value=height_gi, continuous_update=True)
            solara.InputFloat("Percentage of area [%]", value=percarea_gi, continuous_update=True)
            solara.use_effect(lambda: update_draw_tools(), [measure_type.value])
        elif measure_type.value == "Water storage":
            solara.InputFloat("Volume [m³]", value=volume_wst, continuous_update=True)
            solara.use_effect(lambda: update_draw_tools(), [measure_type.value])
        elif measure_type.value == "Elevate properties":
            solara.InputFloat("Elevation [m]", value=elevation_prop, continuous_update=True)
            solara.use_effect(lambda: update_draw_tools(), [measure_type.value])
        elif measure_type.value == "Floodproof properties":
            solara.InputFloat("Elevation [m]", value=elevation_fpprop, continuous_update=True)
            solara.use_effect(lambda: update_draw_tools(), [measure_type.value])
        elif measure_type.value == "Buyout properties":
            solara.use_effect(lambda: update_draw_tools(), [measure_type.value])

        solara.Markdown("Click on the map to place the measure location (or draw a shape).")
        if measures_list.value:
            selected_measures = [f"{measure['type']}, '{measure['name']}'" for measure in measures_list.value]
            formatted_measures = "\n".join(selected_measures)
            solara.Markdown(f"**Selected Measures:**\n{formatted_measures}")

        if error_message.value:
            solara.Markdown(error_message.value, style={"color": "red"})

        with solara.Row():
            solara.Button("Save Current and Add New Measure", on_click=save_current_add_new, style={"margin-top": "20px", "margin-left": "-20px"})
            solara.Button("Clear All", on_click=clear_measures, style={"margin-top": "80px", "margin-left": "-348px"})
            solara.Button("Save Strategy", on_click=save_strategy, style={"margin-top": "80px", "margin-left": "54px"})

####################################### TAB 4 SAVE #######################################
scenarioName = solara.reactive("Scenario Name")

@solara.component
def SaveRun():

    def save_and_run():
        global scenario
        scenario = {
            'name': scenarioName.value, # "test_scenario",
            'event': event_dict,
            'projection': proj_dict,
            'strategy': strategy,
        }
        # Write toml file
        # scenario_fn = fa_database / f"{scenario['name']}_toplevel.toml"
        # with open(scenario_fn, 'wb+') as f:
        #     tomli_w.dump(scenario, f)

        # LAUNCH WORKFLOW!!!
        # database_fn = Path('/home/wotromp/InterTwin/FloodAdapt_database/Humber')
        # scenario_name = 'test_scenario'
        # create_workflow_config(database_fn, scenario_name)
        # run_fa_scenario_workflow(database_fn, scenario_name)

    with solara.Card("Save inputs and Run model", style={"width": "100%", "padding": "10px"}):
        solara.InputText("Scenario Name", value=scenarioName, continuous_update=True)
        solara.Markdown(f"**Your Scenario Name**: {scenarioName.value}")
        
        solara.Button("Save & Run", on_click=save_and_run)



##############################################################################################


@solara.component
def SettingsTabs():
    with solara.Row():
        solara.Button("Event", on_click=lambda: selected_tab.set('Event'))
        solara.Button("Projections", on_click=lambda: selected_tab.set('Projections'))
        solara.Button("Measures", on_click=lambda: selected_tab.set('Measures'))
        solara.Button("Save & Run", on_click=lambda: selected_tab.set('SaveRun'))

    if selected_tab.value == 'Event':
        TabEvent()
    elif selected_tab.value == 'Projections':
        TabProjections()
    elif selected_tab.value == 'Measures':
        TabMeasures()
    elif selected_tab.value == 'SaveRun':
        SaveRun()


@solara.component
def App():
    with solara.Columns():
        with solara.Column(style={"width": "70%", "min-width": "500px"}):
            solara.use_memo(lambda: display(m), [])
        with solara.Column(style={"width": "30%", "min-width": "400px"}):
            SettingsTabs()

App()

In [ ]:
event_dict


In [ ]:
proj_dict

In [ ]:
measures_list.value

In [ ]:
strategy

# Copy from code from top, with seperate functions

## slight adjustments, no overall improvement of schema

In [9]:
# >>>>>>>>>>>>>>>> FUNCTIONS ONLY <<<<<<<<<<<<<<<<<<<<<

############################################## GENERAL ############################################
def handle_draw(target, action, geo_json):
    print(f"Action: {action}, GeoJSON: {geo_json}") 
    if action == 'created' and 'geometry' in geo_json:
        geo_type = geo_json['geometry']['type']
        coords = geo_json['geometry']['coordinates']
        
        if current_geometry_type.value and current_geometry_type.value != geo_type:
            error_message.value = f"**Error**: You cannot draw a {geo_type} after a {current_geometry_type.value}. Please clear the previous geometry before adding a new type."
            return
        
        if geo_type == 'Polygon':
            polygon_coords = coords[0]
            selected_geometry.value = {'type': 'Polygon', 'coordinates': polygon_coords}
            drawn_polygons.value.append({'type': 'Polygon', 'coordinates': polygon_coords})
            print("Polygon coordinates updated:", selected_geometry.value)
        
        elif geo_type == 'LineString':  # Handling polylines
            selected_geometry.value = {'type': 'LineString', 'coordinates': coords}
            drawn_polylines.value.append({'type': 'LineString', 'coordinates': coords})
            print("Polyline coordinates updated:", selected_geometry.value)

        elif geo_type == 'Point':  # Handling markers
            if point_placed.value:
                error_message.value = "**Error**: Only one point can be placed. Please first save the measure before adding another one."
                return
            selected_geometry.value = {'type': 'Point', 'coordinates': coords}
            drawn_markers.value.append({'type': 'Point', 'coordinates': coords})
            point_placed.set(True)  # Set point placed flag to True
            print("Marker coordinates updated:", selected_geometry.value)


        current_geometry_type.value = geo_type  # Update the current geometry type
        error_message.value = ""


############################################## TAB 1 ##############################################

# def save_inputs():
#         event_dict["name"] = eventName.value
#         event_dict["start_time"] = start_date.value.strftime('%Y-%m-%d %H:%M:%S')
#         event_dict["end_time"] = end_date.value.strftime('%Y-%m-%d %H:%M:%S')
#         event_dict["data_catalogues"] = 'MISSING'
#         event_dict["sfincs_forcing"] = {'meteo': 'MISSING', 'waterlevel': 'MISSING'}
def save_inputs(event_name, start_date, end_date):
    global event_dict
    event_dict["name"] = event_name
    event_dict["start_time"] = start_date.strftime('%Y-%m-%d %H:%M:%S')
    event_dict["end_time"] = end_date.strftime('%Y-%m-%d %H:%M:%S')
    event_dict["data_catalogues"] = 'MISSING'
    event_dict["sfincs_forcing"] = {'meteo': 'MISSING', 'waterlevel': 'MISSING'}

############################################## TAB 2 ##############################################

def save_inputs_projections():
        proj_dict["name"] = projName.value
        proj_dict["physical_projection"] = {'sea_level_rise': SLR_input.value,
                                            'rainfall_increase': RAIN_input.value}
        proj_dict["socio_economic_change"] = {'population_growth_existing': POP_input.value,
                                            'economic_growth': ECON_input.value}


############################################## TAB 3 ##############################################


def clear_all_drawn_geometries():
    global draw_control
    m.remove_control(draw_control)
    draw_control = DrawControl(
        polyline={'shapeOptions': {'color': 'blue', 'weight': 4}},
        polygon={'shapeOptions': {'color': 'red', 'weight': 4}},
        marker={},
        rectangle={},
        circle={},    )
    draw_control.on_draw(handle_draw)
    m.add_control(draw_control)
    drawn_polylines.set([])
    drawn_polygons.set([])
    drawn_markers.set([])
    point_placed.set(False) 

def update_draw_tools():
    print(f"Updating draw tools for measure_type: {measure_type.value}")
    global draw_control
    m.remove_control(draw_control)
    
    if measure_type.value == "Floodwall":
        new_draw_control = DrawControl(polyline={'shapeOptions': {'color': 'blue', 'weight': 4}},polygon={},circlemarker={})
    elif measure_type.value == "Pump":
        new_draw_control = DrawControl(polyline={},polygon={},marker={})  # Enable marker only
    elif measure_type.value in ["Water square", "Green infrastructure", "Water storage", "Elevate properties", "Floodproof properties", "Buyout properties"]:
        new_draw_control = DrawControl(polyline={},polygon={'shapeOptions': {'color': 'red', 'weight': 4}},circlemarker={})

    draw_control = new_draw_control
    draw_control.on_draw(handle_draw)
    m.add_control(draw_control)

    drawn_polylines.set([])
    drawn_polygons.set([])
    drawn_markers.set([])
    point_placed.set(False)

def save_current_add_new():
        if selected_geometry.value is None:
            error_message.value = "**Error**: Please place the measure on the map."
            return
        else:
            error_message.set("")
        
        measure_misc  = {}
        if measure_type.value == "Floodwall":
            mtype = "floodwall"
            measure_misc = {"elevation": {"value": elevation_fw.value, "units": 'meters'}}
        elif measure_type.value == "Pump":
            mtype = "pump"
            measure_misc = {"discharge": {"value": discharge_fw.value, "units": 'm3/s'}}
        elif measure_type.value == "Water square":
            mtype = "water_square"
            measure_misc = {"volume": {"value": volume_ws.value, "units": 'm3'},
                            "height": {"value": height_ws.value, "units": "meters"}}
        elif measure_type.value == "Green infrastructure":
            mtype = "greening"
            measure_misc = {"volume": {"value": volume_gi.value, "units": 'm3'},
                            "height": {"value": height_gi.value, "units": "meters"},
                            "percent_area": percarea_gi.value}
        elif measure_type.value == "Water storage":
            mtype = "total_storage"
            measure_misc = {"volume": {"value": volume_wst.value, "units": 'm3'}}
        elif measure_type.value == "Elevate properties":
            mtype = "elevate_properties"
            measure_misc = {"elevation": {"value": elevation_prop.value, "units": 'meters'}}
        elif measure_type.value == "Floodproof properties":
            mtype = "floodproof_properties"
            measure_misc = {"elevation": {"value": elevation_fpprop.value, "units": 'meters'}}
        elif measure_type.value == "Buyout properties":
            mtype = "buyout_properties"
            measure_misc = {} # no additional properties

        measure = {
            "name": measureName.value,
            "type": mtype, # measure_type.value,
            "selection": selected_geometry.value,
            "misc": measure_misc        }
        measures_list.value.append(measure)
        save_directory = Path.cwd() # Path(r"/mnt/c/Users/santjer/OneDrive - Stichting Deltares/Documents/DestinE/Technical/02_Solara")
        geojson_data = {"measure_area": selected_geometry.value}
        geojson_filename = save_directory / f"{measureName.value}_geometry.geojson"
        with open(geojson_filename, 'w') as geojson_file:
            json.dump(geojson_data, geojson_file)
        measure["selection"] = str(geojson_filename) # - - - - - - - - - - - - - - - - - - - - - - - -
        clear_all_drawn_geometries()
        selected_geometry.set(None)
        current_geometry_type.set(None)
        measureName.set("Measure Name")
        measure_type.set("Floodwall")
        elevation_fw.set(0)
        discharge_fw.set(0) 
        volume_ws.set(0) 
        height_ws.set(0) 
        volume_gi.set(0) 
        height_gi.set(0) 
        percarea_gi.set(0)
        volume_wst.set(0) 
        elevation_prop.set(0) 
        elevation_fpprop.set(0)

def save_all_measures():
    global strategy
    error_message.set("")
    clear_all_drawn_geometries()
    selected_geometry.set(None)
    current_geometry_type.set(None)
    
    strategy = {'name': strategyName.value}
    for idx, measure in enumerate(measures_list.value, start=1):
        if measure['selection'] is None:
            continue
        measure_name = f"measure{idx}"
        file_path = Path.cwd() / f"{measure['name']}_geometry.geojson"# f"{measureName.value}_geometry.geojson" #####################
        strategy[measure_name] = {
            'name': measure['name'],
            'type': measure['type'].lower(),  # Assuming the type should be lowercase
            'selection': str(file_path), # str(geojson_filename),
            'misc': measure['misc']}
    strategyName.set("Strategy Name")

def clear_measures():
    global strategy
    for measure in measures_list.value:
        measure_name = measure['name']
        file_path = Path.cwd() / f"{measure_name}_geometry.geojson"
        if file_path.exists():
            file_path.unlink()
    measures_list.set([]) 
    selected_geometry.set(None)
    current_geometry_type.set(None)
    error_message.set("")
    clear_all_drawn_geometries()
    measureName.set("Measure Name")
    measure_type.set("Floodwall")
    strategyName.set("Strategy Name")
    strategy = {} 

def save_strategy():
    save_current_add_new()
    save_all_measures()
###################################################################################################


In [ ]:
# >>>>>> CORRECT BEST VERSION OF CODE BELOW \_v_/ without functions (find them on top) <<<<<<<<<<<<<<<<<<<

polyline_coordinates = solara.reactive([])
polygon_coordinates = solara.reactive([])
marker_coordinates = solara.reactive([])
drawn_polylines = solara.reactive([])
drawn_polygons = solara.reactive([])
drawn_markers = solara.reactive([])
selected_geometry = solara.reactive(None) 
selected_tab = solara.reactive('Event') 
measures_list = solara.reactive([])
error_message = solara.reactive("")
current_geometry_type = solara.reactive(None)  # Track current geometry type
point_placed = solara.reactive(False)
open_delete_confirmation = solara.reactive(False)

m = Map(center=(52.08654741528378, 4.295223531699989), zoom=10, scroll_wheel_zoom=True, basemap=basemaps.OpenStreetMap.Mapnik)

draw_control = DrawControl(
    polyline={'shapeOptions': {'color': 'blue', 'weight': 4}},
    polygon={'shapeOptions': {'color': 'red', 'weight': 4}},
    marker={},  # Allow marker drawing
    rectangle={},  # Disables rectangle drawing
    circle={},  # Disables circle drawing
)

####################################### FUNCTION handle_draw #######################################

draw_control.on_draw(handle_draw)
m.add_control(draw_control)


map_output = Output()
with map_output:
    display(m)


####################################### TAB 1 WEATHER EVENT #######################################
event_dict = {}

@solara.component
def TabEvent():
    # global selected_start_date, selected_end_date
    start_date = solara.use_reactive(dt.date.today())
    end_date = solara.use_reactive(dt.date.today())
    # selected_start_date = start_date.value
    # selected_end_date = end_date.value
    eventName = solara.use_reactive("Event Name")

    
	####################################### FUNCTION save_inputs #######################################
    
    with solara.Card("Configure the Weather Event", style={"width": "100%", "padding": "10px"}):
        solara.InputText("Event Name", value=eventName, continuous_update=True)
        solara.Markdown(f"**Your Event Name**: {eventName.value}")
        
        solara.Text("Select the Start Date:")
        solara.lab.InputDate(start_date)
        
        solara.Text("Select the End Date:")
        solara.lab.InputDate(end_date)
        
        if end_date.value < start_date.value:
            solara.Markdown("**Warning**: The end date cannot be earlier than the start date.", style={"color": "red"})
        
        solara.Button("Save Inputs", on_click=lambda: save_inputs(eventName.value, start_date.value, end_date.value))


####################################### TAB 2 PROJECTIONS #######################################
SLR_input = solara.reactive(0.0)
RAIN_input = solara.reactive(0.0)
POP_input = solara.reactive(0.0)
ECON_input = solara.reactive(0.0)
projName = solara.reactive("Projection Name")
proj_dict = {}

@solara.component
def TabProjections():
    ####################################### FUNCTION save_inputs_projections #######################################

    with solara.Card("Projections", style={"width": "100%", "padding": "10px"}):
        solara.InputText("Projection Name", value=projName, continuous_update=True)
        solara.Markdown(f"**Your Projection Name**: {projName.value}")
        solara.InputFloat("Sea Level Rise (metres)", value=SLR_input, continuous_update=True)
        solara.InputFloat("Precipitation Increase (%)", value=RAIN_input, continuous_update=True)
        solara.InputFloat("Population Growth (%)", value=POP_input, continuous_update=True)
        solara.InputFloat("Economic Growth (%)", value=ECON_input, continuous_update=True)
        
        solara.Button("Save Inputs", on_click=save_inputs_projections, style={"margin-top": "20px"})


####################################### TAB 3 MEASURES #######################################
strategyName = solara.reactive("Strategy Name")
measureName = solara.reactive("Measure Name")
measure_types = ["Floodwall","Pump","Water square","Green infrastructure","Water storage",
                 "Elevate properties","Floodproof properties","Buyout properties"]
measure_type = solara.reactive("Floodwall")
elevation_fw = solara.reactive(0)  # For floodwall 
discharge_fw = solara.reactive(0)  # For pump 
volume_ws = solara.reactive(0)  # For water square
height_ws = solara.reactive(0)  # For water square 
volume_gi = solara.reactive(0)  # For green infrastructure
height_gi = solara.reactive(0)  # For green infrastructure
percarea_gi = solara.reactive(0)  # For green infrastructure 
volume_wst = solara.reactive(0)  # For water storage 
elevation_prop = solara.reactive(0)  # For properties 
elevation_fpprop = solara.reactive(0)  # For floodproof properties 
strategy = {} 

####################################### FUNCTION clear_all_drawn_geometries #######################################

####################################### FUNCTION update_draw_tools #######################################

####################################### FUNCTION save_current_add_new #######################################

####################################### FUNCTION save_all_measures #######################################

####################################### FUNCTION clear_measures #######################################

@solara.component
def TabMeasures():  

    with solara.Card("Strategy Name", style={"width": "100%", "padding": "10px"}):
        solara.InputText("Strategy Name", value=strategyName, continuous_update=True)
        
    with solara.Card("Measures", style={"width": "100%", "padding": "10px"}):
        solara.InputText("Measure Name", value=measureName, continuous_update=True)
        solara.Markdown(f"**Your Measure Name**: {measureName.value}")

        solara.Select(label="Measure Type", value=measure_type, values=measure_types)

        if measure_type.value == "Floodwall":
            solara.InputFloat("Elevation [m]", value=elevation_fw, continuous_update=True)
            solara.use_effect(lambda: update_draw_tools(), [measure_type.value])
        elif measure_type.value == "Pump":
            solara.InputFloat("Discharge [m³/s]", value=discharge_fw, continuous_update=True)
            solara.use_effect(lambda: update_draw_tools(), [measure_type.value])
        elif measure_type.value == "Water square":
            solara.InputFloat("Volume [m³]", value=volume_ws, continuous_update=True)
            solara.InputFloat("Height [m]", value=height_ws, continuous_update=True)
            solara.use_effect(lambda: update_draw_tools(), [measure_type.value])
        elif measure_type.value == "Green infrastructure":
            solara.InputFloat("Volume [m³]", value=volume_gi, continuous_update=True)
            solara.InputFloat("Height [m]", value=height_gi, continuous_update=True)
            solara.InputFloat("Percentage of area [%]", value=percarea_gi, continuous_update=True)
            solara.use_effect(lambda: update_draw_tools(), [measure_type.value])
        elif measure_type.value == "Water storage":
            solara.InputFloat("Volume [m³]", value=volume_wst, continuous_update=True)
            solara.use_effect(lambda: update_draw_tools(), [measure_type.value])
        elif measure_type.value == "Elevate properties":
            solara.InputFloat("Elevation [m]", value=elevation_prop, continuous_update=True)
            solara.use_effect(lambda: update_draw_tools(), [measure_type.value])
        elif measure_type.value == "Floodproof properties":
            solara.InputFloat("Elevation [m]", value=elevation_fpprop, continuous_update=True)
            solara.use_effect(lambda: update_draw_tools(), [measure_type.value])
        elif measure_type.value == "Buyout properties":
            solara.use_effect(lambda: update_draw_tools(), [measure_type.value])

        solara.Markdown("Click on the map to place the measure location (or draw a shape).")
        if measures_list.value:
            selected_measures = [f"{measure['type']}, '{measure['name']}'" for measure in measures_list.value]
            formatted_measures = "\n".join(selected_measures)
            solara.Markdown(f"**Selected Measures:**\n{formatted_measures}")

        if error_message.value:
            solara.Markdown(error_message.value, style={"color": "red"})

        with solara.Row():
            solara.Button("Save Current and Add New Measure", on_click=save_current_add_new, style={"margin-top": "20px", "margin-left": "-20px"})
            solara.Button("Clear All", on_click=clear_measures, style={"margin-top": "80px", "margin-left": "-348px"})
            solara.Button("Save Strategy", on_click=save_strategy, style={"margin-top": "80px", "margin-left": "54px"})


####################################### TAB 4 SAVE #######################################
scenarioName = solara.reactive("Scenario Name")

@solara.component
def SaveRun():

    def save_and_run():
        global scenario
        scenario = {
            'name': scenarioName.value, # "test_scenario",
            'event': event_dict,
            'projection': proj_dict,
            'strategy': strategy,
        }
        # Write toml file
        # scenario_fn = fa_database / f"{scenario['name']}_toplevel.toml"
        # with open(scenario_fn, 'wb+') as f:
        #     tomli_w.dump(scenario, f)

        # LAUNCH WORKFLOW!!!
        # database_fn = Path('/home/wotromp/InterTwin/FloodAdapt_database/Humber')
        # scenario_name = 'test_scenario'
        # create_workflow_config(database_fn, scenario_name)
        # run_fa_scenario_workflow(database_fn, scenario_name)

    with solara.Card("Save inputs and Run model", style={"width": "100%", "padding": "10px"}):
        solara.InputText("Scenario Name", value=scenarioName, continuous_update=True)
        solara.Markdown(f"**Your Scenario Name**: {scenarioName.value}")
        
        solara.Button("Save & Run", on_click=save_and_run)


##############################################################################################

@solara.component
def SettingsTabs():
    with solara.Row():
        solara.Button("Event", on_click=lambda: selected_tab.set('Event'))
        solara.Button("Projections", on_click=lambda: selected_tab.set('Projections'))
        solara.Button("Measures", on_click=lambda: selected_tab.set('Measures'))
        solara.Button("Save & Run", on_click=lambda: selected_tab.set('SaveRun'))

    if selected_tab.value == 'Event':
        TabEvent()
    elif selected_tab.value == 'Projections':
        TabProjections()
    elif selected_tab.value == 'Measures':
        TabMeasures()
    elif selected_tab.value == 'SaveRun':
        SaveRun()


@solara.component
def App():
    with solara.Columns():
        with solara.Column(style={"width": "70%", "min-width": "500px"}):
            solara.use_memo(lambda: display(m), [])
        with solara.Column(style={"width": "30%", "min-width": "400px"}):
            SettingsTabs()

App()


In [ ]:
measures_list.value

In [ ]:
strategy

# improve on run button

In [17]:
# >>>>>>>>>>>>>>>> FUNCTIONS ONLY <<<<<<<<<<<<<<<<<<<<<

############################################## GENERAL ############################################
def handle_draw(target, action, geo_json):
    print(f"Action: {action}, GeoJSON: {geo_json}") 
    if action == 'created' and 'geometry' in geo_json:
        geo_type = geo_json['geometry']['type']
        coords = geo_json['geometry']['coordinates']
        
        if current_geometry_type.value and current_geometry_type.value != geo_type:
            error_message.value = f"**Error**: You cannot draw a {geo_type} after a {current_geometry_type.value}. Please clear the previous geometry before adding a new type."
            return
        
        if geo_type == 'Polygon':
            polygon_coords = coords[0]
            selected_geometry.value = {'type': 'Polygon', 'coordinates': polygon_coords}
            drawn_polygons.value.append({'type': 'Polygon', 'coordinates': polygon_coords})
            print("Polygon coordinates updated:", selected_geometry.value)
        
        elif geo_type == 'LineString':  # Handling polylines
            selected_geometry.value = {'type': 'LineString', 'coordinates': coords}
            drawn_polylines.value.append({'type': 'LineString', 'coordinates': coords})
            print("Polyline coordinates updated:", selected_geometry.value)

        elif geo_type == 'Point':  # Handling markers
            if point_placed.value:
                error_message.value = "**Error**: Only one point can be placed. Please first save the measure before adding another one."
                return
            selected_geometry.value = {'type': 'Point', 'coordinates': coords}
            drawn_markers.value.append({'type': 'Point', 'coordinates': coords})
            point_placed.set(True)  # Set point placed flag to True
            print("Marker coordinates updated:", selected_geometry.value)


        current_geometry_type.value = geo_type  # Update the current geometry type
        error_message.value = ""


############################################## TAB 1 ##############################################

# def save_inputs():
#         event_dict["name"] = eventName.value
#         event_dict["start_time"] = start_date.value.strftime('%Y-%m-%d %H:%M:%S')
#         event_dict["end_time"] = end_date.value.strftime('%Y-%m-%d %H:%M:%S')
#         event_dict["data_catalogues"] = 'MISSING'
#         event_dict["sfincs_forcing"] = {'meteo': 'MISSING', 'waterlevel': 'MISSING'}
def save_inputs(event_name, start_date, end_date):
    global event_dict
    event_dict["name"] = event_name
    event_dict["start_time"] = start_date.strftime('%Y-%m-%d %H:%M:%S')
    event_dict["end_time"] = end_date.strftime('%Y-%m-%d %H:%M:%S')
    event_dict["data_catalogues"] = 'MISSING'
    event_dict["sfincs_forcing"] = {'meteo': 'MISSING', 'waterlevel': 'MISSING'}

############################################## TAB 2 ##############################################

def save_inputs_projections():
        proj_dict["name"] = projName.value
        proj_dict["physical_projection"] = {'sea_level_rise': SLR_input.value,
                                            'rainfall_increase': RAIN_input.value}
        proj_dict["socio_economic_change"] = {'population_growth_existing': POP_input.value,
                                            'economic_growth': ECON_input.value}


############################################## TAB 3 ##############################################


def clear_all_drawn_geometries():
    global draw_control
    m.remove_control(draw_control)
    draw_control = DrawControl(
        polyline={'shapeOptions': {'color': 'blue', 'weight': 4}},
        polygon={'shapeOptions': {'color': 'red', 'weight': 4}},
        marker={},
        rectangle={},
        circle={},    )
    draw_control.on_draw(handle_draw)
    m.add_control(draw_control)
    drawn_polylines.set([])
    drawn_polygons.set([])
    drawn_markers.set([])
    point_placed.set(False) 

def update_draw_tools():
    print(f"Updating draw tools for measure_type: {measure_type.value}")
    global draw_control
    m.remove_control(draw_control)
    
    if measure_type.value == "Floodwall":
        new_draw_control = DrawControl(polyline={'shapeOptions': {'color': 'blue', 'weight': 4}},polygon={},circlemarker={})
    elif measure_type.value == "Pump":
        new_draw_control = DrawControl(polyline={},polygon={},marker={})  # Enable marker only
    elif measure_type.value in ["Water square", "Green infrastructure", "Water storage", "Elevate properties", "Floodproof properties", "Buyout properties"]:
        new_draw_control = DrawControl(polyline={},polygon={'shapeOptions': {'color': 'red', 'weight': 4}},circlemarker={})

    draw_control = new_draw_control
    draw_control.on_draw(handle_draw)
    m.add_control(draw_control)

    drawn_polylines.set([])
    drawn_polygons.set([])
    drawn_markers.set([])
    point_placed.set(False)

def save_current_add_new():
        if selected_geometry.value is None:
            error_message.value = "**Error**: Please place the measure on the map."
            return
        else:
            error_message.set("")
        
        measure_misc  = {}
        if measure_type.value == "Floodwall":
            mtype = "floodwall"
            measure_misc = {"elevation": {"value": elevation_fw.value, "units": 'meters'}}
        elif measure_type.value == "Pump":
            mtype = "pump"
            measure_misc = {"discharge": {"value": discharge_fw.value, "units": 'm3/s'}}
        elif measure_type.value == "Water square":
            mtype = "water_square"
            measure_misc = {"volume": {"value": volume_ws.value, "units": 'm3'},
                            "height": {"value": height_ws.value, "units": "meters"}}
        elif measure_type.value == "Green infrastructure":
            mtype = "greening"
            measure_misc = {"volume": {"value": volume_gi.value, "units": 'm3'},
                            "height": {"value": height_gi.value, "units": "meters"},
                            "percent_area": percarea_gi.value}
        elif measure_type.value == "Water storage":
            mtype = "total_storage"
            measure_misc = {"volume": {"value": volume_wst.value, "units": 'm3'}}
        elif measure_type.value == "Elevate properties":
            mtype = "elevate_properties"
            measure_misc = {"elevation": {"value": elevation_prop.value, "units": 'meters'}}
        elif measure_type.value == "Floodproof properties":
            mtype = "floodproof_properties"
            measure_misc = {"elevation": {"value": elevation_fpprop.value, "units": 'meters'}}
        elif measure_type.value == "Buyout properties":
            mtype = "buyout_properties"
            measure_misc = {} # no additional properties

        measure = {
            "name": measureName.value,
            "type": mtype, # measure_type.value,
            "selection": selected_geometry.value,
            "misc": measure_misc        }
        measures_list.value.append(measure)
        save_directory = Path.cwd() # Path(r"/mnt/c/Users/santjer/OneDrive - Stichting Deltares/Documents/DestinE/Technical/02_Solara")
        geojson_data = {"measure_area": selected_geometry.value}
        geojson_filename = save_directory / f"{measureName.value}_geometry.geojson"
        with open(geojson_filename, 'w') as geojson_file:
            json.dump(geojson_data, geojson_file)
        measure["selection"] = str(geojson_filename) # - - - - - - - - - - - - - - - - - - - - - - - -
        clear_all_drawn_geometries()
        selected_geometry.set(None)
        current_geometry_type.set(None)
        measureName.set("Measure Name")
        measure_type.set("Floodwall")
        elevation_fw.set(0)
        discharge_fw.set(0) 
        volume_ws.set(0) 
        height_ws.set(0) 
        volume_gi.set(0) 
        height_gi.set(0) 
        percarea_gi.set(0)
        volume_wst.set(0) 
        elevation_prop.set(0) 
        elevation_fpprop.set(0)

def save_all_measures():
    global strategy
    error_message.set("")
    clear_all_drawn_geometries()
    # save_current_add_new()
    selected_geometry.set(None)
    current_geometry_type.set(None)
    
    strategy = {'name': strategyName.value}
    for idx, measure in enumerate(measures_list.value, start=1):
        if measure['selection'] is None:
            continue
        measure_name = f"measure{idx}"
        file_path = Path.cwd() / f"{measure['name']}_geometry.geojson"# f"{measureName.value}_geometry.geojson" #####################
        strategy[measure_name] = {
            'name': measure['name'],
            'type': measure['type'].lower(),  # Assuming the type should be lowercase
            'selection': str(file_path), # str(geojson_filename),
            'misc': measure['misc']}
    strategyName.set("Strategy Name")

def clear_measures():
    global strategy
    for measure in measures_list.value:
        measure_name = measure['name']
        file_path = Path.cwd() / f"{measure_name}_geometry.geojson"
        if file_path.exists():
            file_path.unlink()
    measures_list.set([]) 
    selected_geometry.set(None)
    current_geometry_type.set(None)
    error_message.set("")
    clear_all_drawn_geometries()
    measureName.set("Measure Name")
    measure_type.set("Floodwall")
    strategyName.set("Strategy Name")
    strategy = {} 


def save_strategy():
    save_current_add_new()
    save_all_measures()

###################################################################################################


In [ ]:
# >>>>>> CORRECT BEST VERSION OF CODE BELOW \_v_/ without functions (find them on top) <<<<<<<<<<<<<<<<<<<

polyline_coordinates = solara.reactive([])
polygon_coordinates = solara.reactive([])
marker_coordinates = solara.reactive([])
drawn_polylines = solara.reactive([])
drawn_polygons = solara.reactive([])
drawn_markers = solara.reactive([])
selected_geometry = solara.reactive(None) 
selected_tab = solara.reactive('Event') 
measures_list = solara.reactive([])
error_message = solara.reactive("")
current_geometry_type = solara.reactive(None)  # Track current geometry type
point_placed = solara.reactive(False)
open_delete_confirmation = solara.reactive(False)

m = Map(center=(52.08654741528378, 4.295223531699989), zoom=10, scroll_wheel_zoom=True, basemap=basemaps.OpenStreetMap.Mapnik)

draw_control = DrawControl(
    polyline={'shapeOptions': {'color': 'blue', 'weight': 4}},
    polygon={'shapeOptions': {'color': 'red', 'weight': 4}},
    marker={},  # Allow marker drawing
    rectangle={},  # Disables rectangle drawing
    circle={},  # Disables circle drawing
)

####################################### FUNCTION handle_draw #######################################

draw_control.on_draw(handle_draw)
m.add_control(draw_control)


map_output = Output()
with map_output:
    display(m)


####################################### TAB 1 WEATHER EVENT #######################################
event_dict = {}

@solara.component
def TabEvent():
    # global selected_start_date, selected_end_date
    start_date = solara.use_reactive(dt.date.today())
    end_date = solara.use_reactive(dt.date.today())
    # selected_start_date = start_date.value
    # selected_end_date = end_date.value
    eventName = solara.use_reactive("Event Name")

    
	####################################### FUNCTION save_inputs #######################################
    
    with solara.Card("Configure the Weather Event", style={"width": "100%", "padding": "10px"}):
        solara.InputText("Event Name", value=eventName, continuous_update=True)
        solara.Markdown(f"**Your Event Name**: {eventName.value}")
        
        solara.Text("Select the Start Date:")
        solara.lab.InputDate(start_date)
        
        solara.Text("Select the End Date:")
        solara.lab.InputDate(end_date)
        
        if end_date.value < start_date.value:
            solara.Markdown("**Warning**: The end date cannot be earlier than the start date.", style={"color": "red"})
        
        solara.Button("Save Inputs", on_click=lambda: save_inputs(eventName.value, start_date.value, end_date.value))


####################################### TAB 2 PROJECTIONS #######################################
SLR_input = solara.reactive(0.0)
RAIN_input = solara.reactive(0.0)
POP_input = solara.reactive(0.0)
ECON_input = solara.reactive(0.0)
projName = solara.reactive("Projection Name")
proj_dict = {}

@solara.component
def TabProjections():
    ####################################### FUNCTION save_inputs_projections #######################################

    with solara.Card("Projections", style={"width": "100%", "padding": "10px"}):
        solara.InputText("Projection Name", value=projName, continuous_update=True)
        solara.Markdown(f"**Your Projection Name**: {projName.value}")
        solara.InputFloat("Sea Level Rise (metres)", value=SLR_input, continuous_update=True)
        solara.InputFloat("Precipitation Increase (%)", value=RAIN_input, continuous_update=True)
        solara.InputFloat("Population Growth (%)", value=POP_input, continuous_update=True)
        solara.InputFloat("Economic Growth (%)", value=ECON_input, continuous_update=True)
        
        solara.Button("Save Inputs", on_click=save_inputs_projections, style={"margin-top": "20px"})


####################################### TAB 3 MEASURES #######################################
strategyName = solara.reactive("Strategy Name")
measureName = solara.reactive("Measure Name")
measure_types = ["Floodwall","Pump","Water square","Green infrastructure","Water storage",
                 "Elevate properties","Floodproof properties","Buyout properties"]
measure_type = solara.reactive("Floodwall")
elevation_fw = solara.reactive(0)  # For floodwall 
discharge_fw = solara.reactive(0)  # For pump 
volume_ws = solara.reactive(0)  # For water square
height_ws = solara.reactive(0)  # For water square 
volume_gi = solara.reactive(0)  # For green infrastructure
height_gi = solara.reactive(0)  # For green infrastructure
percarea_gi = solara.reactive(0)  # For green infrastructure 
volume_wst = solara.reactive(0)  # For water storage 
elevation_prop = solara.reactive(0)  # For properties 
elevation_fpprop = solara.reactive(0)  # For floodproof properties 
strategy = {} 

####################################### FUNCTION clear_all_drawn_geometries #######################################

####################################### FUNCTION update_draw_tools #######################################

####################################### FUNCTION save_current_add_new #######################################

####################################### FUNCTION save_all_measures #######################################

####################################### FUNCTION clear_measures #######################################

@solara.component
def TabMeasures():  

    with solara.Card("Strategy Name", style={"width": "100%", "padding": "10px"}):
        solara.InputText("Strategy Name", value=strategyName, continuous_update=True)
        
    with solara.Card("Measures", style={"width": "100%", "padding": "10px"}):
        solara.InputText("Measure Name", value=measureName, continuous_update=True)
        solara.Markdown(f"**Your Measure Name**: {measureName.value}")

        solara.Select(label="Measure Type", value=measure_type, values=measure_types)

        if measure_type.value == "Floodwall":
            solara.InputFloat("Elevation [m]", value=elevation_fw, continuous_update=True)
            solara.use_effect(lambda: update_draw_tools(), [measure_type.value])
        elif measure_type.value == "Pump":
            solara.InputFloat("Discharge [m³/s]", value=discharge_fw, continuous_update=True)
            solara.use_effect(lambda: update_draw_tools(), [measure_type.value])
        elif measure_type.value == "Water square":
            solara.InputFloat("Volume [m³]", value=volume_ws, continuous_update=True)
            solara.InputFloat("Height [m]", value=height_ws, continuous_update=True)
            solara.use_effect(lambda: update_draw_tools(), [measure_type.value])
        elif measure_type.value == "Green infrastructure":
            solara.InputFloat("Volume [m³]", value=volume_gi, continuous_update=True)
            solara.InputFloat("Height [m]", value=height_gi, continuous_update=True)
            solara.InputFloat("Percentage of area [%]", value=percarea_gi, continuous_update=True)
            solara.use_effect(lambda: update_draw_tools(), [measure_type.value])
        elif measure_type.value == "Water storage":
            solara.InputFloat("Volume [m³]", value=volume_wst, continuous_update=True)
            solara.use_effect(lambda: update_draw_tools(), [measure_type.value])
        elif measure_type.value == "Elevate properties":
            solara.InputFloat("Elevation [m]", value=elevation_prop, continuous_update=True)
            solara.use_effect(lambda: update_draw_tools(), [measure_type.value])
        elif measure_type.value == "Floodproof properties":
            solara.InputFloat("Elevation [m]", value=elevation_fpprop, continuous_update=True)
            solara.use_effect(lambda: update_draw_tools(), [measure_type.value])
        elif measure_type.value == "Buyout properties":
            solara.use_effect(lambda: update_draw_tools(), [measure_type.value])

        solara.Markdown("Click on the map to place the measure location (or draw a shape).")
        if measures_list.value:
            selected_measures = [f"{measure['type']}, '{measure['name']}'" for measure in measures_list.value]
            formatted_measures = "\n".join(selected_measures)
            solara.Markdown(f"**Selected Measures:**\n{formatted_measures}")

        if error_message.value:
            solara.Markdown(error_message.value, style={"color": "red"})

        with solara.Row():
            solara.Button("Save Current and Add New Measure", on_click=save_current_add_new, style={"margin-top": "20px", "margin-left": "-20px"})
            solara.Button("Clear All", on_click=clear_measures, style={"margin-top": "80px", "margin-left": "-348px"})
            solara.Button("Save Strategy", on_click=save_strategy, style={"margin-top": "80px", "margin-left": "54px"})

####################################### TAB 4 SAVE #######################################
scenarioName = solara.reactive("Scenario Name")

@solara.component
def SaveRun():

    def save_and_run():
        global scenario
        scenario = {
            'name': scenarioName.value, # "test_scenario",
            'event': event_dict,
            'projection': proj_dict,
            'strategy': strategy,
        }
        # Write toml file
        # scenario_fn = fa_database / f"{scenario['name']}_toplevel.toml"
        # with open(scenario_fn, 'wb+') as f:
        #     tomli_w.dump(scenario, f)

        # LAUNCH WORKFLOW!!!
        # database_fn = Path('/home/wotromp/InterTwin/FloodAdapt_database/Humber')
        # scenario_name = scenarioName.value # 'test_scenario'
        # create_workflow_config(database_fn, scenario_name)
        # run_fa_scenario_workflow(database_fn, scenario_name)

    with solara.Card("Save inputs and Run model", style={"width": "100%", "padding": "10px"}):
        solara.InputText("Scenario Name", value=scenarioName, continuous_update=True)
        solara.Markdown(f"**Your Scenario Name**: {scenarioName.value}")
        
        solara.Button("Save & Run", on_click=save_and_run)


##############################################################################################
@solara.component
def SettingsTabs():
    with solara.Row():
        solara.Button("Event", on_click=lambda: selected_tab.set('Event'))
        solara.Button("Projections", on_click=lambda: selected_tab.set('Projections'))
        solara.Button("Measures", on_click=lambda: selected_tab.set('Measures'))
        solara.Button("Save & Run", on_click=lambda: selected_tab.set('SaveRun'))

    if selected_tab.value == 'Event':
        TabEvent()
    elif selected_tab.value == 'Projections':
        TabProjections()
    elif selected_tab.value == 'Measures':
        TabMeasures()
    elif selected_tab.value == 'SaveRun':
        SaveRun()


@solara.component
def App():
    with solara.Columns():
        with solara.Column(style={"width": "70%", "min-width": "500px"}):
            solara.use_memo(lambda: display(m), [])
        with solara.Column(style={"width": "30%", "min-width": "400px"}):
            SettingsTabs()

App()


In [ ]:
event_dict

In [ ]:
proj_dict

In [ ]:
strategy

In [ ]:
scenario

In [1]:
from pathlib import Path
from pprint import pprint
import tomli_w

import sys
import os
# Get the path of the DT-FLOOD folder and add it to sys.path
sys.path.append(os.path.abspath(os.path.join('..','..')))

from DT_flood.utils.workflow_utils import create_workflow_config, run_fa_scenario_workflow

ModuleNotFoundError: No module named 'tomli'